In [554]:
import json
import numpy as np
import pandas as pd
import lxml
import re
import os
from datetime import datetime
import pyodbc
import requests

### Get EA script history in JSON format via the API

In [555]:
custom_headers  = {"Authorization": "Gt4qkl1ahFDcFyQwfjMBYT0utbLD4STHQjWdIXzsYH8v3w29M6Tm6tVKI0uMj6Qh", "Content-Type": "application/json"}
url = 'https://ea.executive.automats.app/api/bi/scripts?withHistory=true'

resp = requests.get(url, headers = custom_headers, verify=False)
print(resp.status_code)

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:999: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ea.executive.automats.app'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


200


In [556]:
json_data = json.loads(resp.text)
json_text = json.dumps(json_data, indent=4)

In [557]:
try:
    os.remove('data/execution/try.json')
except:
    print('File does not exist')

with open('data/execution/try.json', 'w') as fw:
    fw.write(json_text)

### Remove invalid characters from the inbound JSON data

In [558]:
try:
    os.remove('data/execution/script_history_fixed.json')
except:
    print('File does not exist')

#with open('data/execution/script_history.json', 'r') as fr:
with open('data/execution/try.json', 'r') as fr:    
    # reading line by line
    lines = fr.readlines()
    last_line = len(lines)
    print(len(lines))
    
    for line in lines:
        line = re.sub("\u0003", "", line)
        with open('data/execution/script_history_fixed.json', 'a') as fw:
            fw.write(line) 

# Previous runtime = 13.8s

1297177


### Parse the multilevel JSON into a CSV file

In [ ]:
f = open('data/execution/script_history_fixed.json')
data = json.loads(f.read())

In [ ]:
try:
    os.remove('data/execution/check_structure.csv')
except:
    print('File does not exist')
with open('data/execution/check_structure.csv', 'a') as fw:
    line = 'topProjectName,name,sessionId,executionLogsId,scriptId,scriptName,script_status,label,step_state,step_status,startTime,endTime,duration,lineNum,StepLogsId,takeScreenshots,screenshotsOnlyOnInterruption\n'
    #line = 'topProjectName,name,sessionId,executionLogsId,scriptId,scriptName,script_status,label,step_state,step_status,startTime,endTime,duration,lineNum,StepLogsId\n'
    fw.write(line)
    i = 0
    for level1 in data:
        for key, value in level1.items():
            #if i == 100: break
            i = i + 1
            takeScreenshots_str = str(level1['takeScreenshots'])
            screenshotsOnlyOnInterruption_str = str(level1['screenshotsOnlyOnInterruption'])            
            try:
                executionLogs_str = json.dumps(level1['executionLogs'])
                executionLogs_json = json.loads(executionLogs_str)
                for level2 in executionLogs_json:
                    for key2, value2 in level2.items():
                        #try:
                        executionStepLogs_str = json.dumps(level2['executionStepLogs'])
                        executionStepLogs_json = json.loads(executionStepLogs_str)
                        if key2 == 'executionStepLogs':
                            records_no_str = str(i)
                            topProjectName = str(level1['topProjectName'])
                            name_str = str(level1['name'])
                            # Remove any commas from name 
                            check = (',' in name_str)
                            if check == True:
                                name_str = re.sub(',', ';', name_str)                            
                            sessionId_str = str(level2['sessionId'])
                            id_str = str(level2['id'])
                            scriptId_str = str(level2['scriptId'])
                            scriptName_str = str(level2['scriptName'])
                            # Remove any commas from scriptName 
                            check = (',' in scriptName_str)
                            if check == True:
                                scriptName_str = re.sub(',', ';', scriptName_str)                             
                            state_str = str(level2['state'])
                            status2_str = str(level2['status'])
                            lastIssues_str = str(level2['lastIssues'])
                            # Remove any commas from lastIssues_str
                            check = (',' in lastIssues_str)
                            if check == True:
                                lastIssues_str = re.sub(',', ';', lastIssues_str)


                            for level3 in executionStepLogs_json:
                                #for key3, value3 in level3.items():
                                status3_str = str(level3['status'])
                                label_str = str(level3['label'])
                                
                                # These lines replace long text test step labels with short text (that does not wrap)
                                check = ('Click Dynamics 365' in label_str)
                                if check == True:
                                    label_str = 'Click Dynamics 365'
                                check = ('Click Sales Hub' in label_str)
                                if check == True:
                                    label_str = 'Click Sales Hub'
                                check = ('Type value function eaExecuteVariableValue()' in label_str)
                                if check == True:
                                    label_str = 'Type value function eaExecuteVariableValue()'
                                check = ('Select from function eaExecuteVariableValue()' in label_str)
                                if check == True:
                                    label_str = 'Select from function eaExecuteVariableValue()'                                    
                                check = ('Select lookup values function eaExecuteVariableValue()' in label_str)
                                if check == True:
                                    label_str = 'Select lookup values function eaExecuteVariableValue()'
                                # Remove any commas from label_str 
                                check = (',' in label_str)
                                if check == True:
                                    label_str = re.sub(',', ';', label_str)
                                # Now replace all newlines that remain in label_str
                                check = ('\n' in label_str)
                                if check == True:
                                    label_str = re.sub('\n', '/nl/', label_str)
                                # Done with replacing text

                                startTime_str = str(level3['startTime'])
                                endTime_str = str(level3['endTime'])
                                duration_str = str(level3['duration'])
                                lineNum_str = str(level3['lineNum'])
                                StepLogs_id_str = str(level3['id'])

                                line = topProjectName+','+name_str+','+sessionId_str+','+id_str+','+scriptId_str+','+scriptName_str+','+status2_str+','+label_str+','+state_str+','+status3_str+','+startTime_str+','+endTime_str+','+duration_str+','+lineNum_str+','+StepLogs_id_str+','+takeScreenshots_str+','+screenshotsOnlyOnInterruption_str+'\n'
                                #line = topProjectName+','+name_str+','+sessionId_str+','+id_str+','+scriptId_str+','+scriptName_str+','+status2_str+','+label_str+','+state_str+','+status3_str+','+startTime_str+','+endTime_str+','+duration_str+','+lineNum_str+','+StepLogs_id_str+'\n'
                                fw.write(line)
                            # except:
                            #     print('Exception on Level2 for Level1 item '+str(i))
            except:
                #print('Exception for Level1 item '+str(i))
                pass
    print('Processed '+str(i)+' items')           

    # Previous runtime = 2m 47.1s     


Processed 10528 items


### Read the denormalized CSV file into a Pandas dataframe

In [ ]:
df = pd.read_csv('data/execution/check_structure.csv')

/tmp/ipykernel_267/2960382116.py:1: DtypeWarning: Columns (12,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/execution/check_structure.csv')


In [ ]:
df.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2496326 entries, 0 to 2496325
Data columns (total 17 columns):
 #   Column                         Non-Null Count    Dtype 
---  ------                         --------------    ----- 
 0   topProjectName                 2496326 non-null  object
 1   name                           2496326 non-null  object
 2   sessionId                      2496326 non-null  object
 3   executionLogsId                2496326 non-null  int64 
 4   scriptId                       2496326 non-null  int64 
 5   scriptName                     2496326 non-null  object
 6   script_status                  2496326 non-null  object
 7   label                          2496326 non-null  object
 8   step_state                     2496326 non-null  object
 9   step_status                    2496326 non-null  object
 10  startTime                      2496326 non-null  object
 11  endTime                        2496326 non-null  object
 12  duration                    

In [ ]:
df.drop_duplicates(keep='first',inplace=True)
df.rename(columns={'label':'StepLabel'}, inplace=True)
df.rename(columns={'name':'testName'}, inplace=True)

In [ ]:
mask = (df['startTime'] == 'None') 
df['startTime'].mask(mask,'2022-01-01T00:00:00.000Z',inplace=True)
df['startTime'] = pd.to_datetime(df['startTime'])

In [ ]:
mask = (df['endTime'] == 'None') 
df['endTime'].mask(mask,'2022-01-01T00:00:00.000Z',inplace=True)
df['endTime'] = pd.to_datetime(df['endTime'])

In [ ]:
mask = (df['duration'] == 'None') 
df['duration'].mask(mask,'0',inplace=True)
df['duration'] = pd.to_numeric(df['duration'])

In [ ]:
df['duration_sec'] = df.duration/1000
df['duration_mins'] = df.duration/1000/60

In [ ]:
df['date'] = pd.to_datetime(df['endTime'], format='%Y-%m-%d').dt.date
df['date'] = pd.to_datetime(df['date'])

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 163935 entries, 0 to 2495685
Data columns (total 20 columns):
 #   Column                         Non-Null Count   Dtype              
---  ------                         --------------   -----              
 0   topProjectName                 163935 non-null  object             
 1   testName                       163935 non-null  object             
 2   sessionId                      163935 non-null  object             
 3   executionLogsId                163935 non-null  int64              
 4   scriptId                       163935 non-null  int64              
 5   scriptName                     163935 non-null  object             
 6   script_status                  163935 non-null  object             
 7   StepLabel                      163935 non-null  object             
 8   step_state                     163935 non-null  object             
 9   step_status                    163935 non-null  object             
 10  startTi

### Filter the dataset for the relevant date(s)

In [ ]:
df2 = df.query("date == '2023-01-26'").copy()

In [ ]:
""" def summary(x):
    runs = x['executionLogsId'].nunique()
    runs = float(runs)
    duration = x['duration'].sum()/1000
    mean_s = x['duration'].mean()/1000
    count = duration/mean_s
    result = {
        'aggr_duration': duration,
        'aggr_duration_minutes': duration/60,
        'runs': runs,
        'mean_sec': duration/runs,
        'mean_minutes': duration/runs/60 ,
        'steps': count
    }
    return pd.Series(result).round(1)

#df2.groupby(['scriptName','sessionId']).apply(summary).to_html('data/execution/result.html')
df2.groupby(['testName', 'executionLogsId']).apply(summary).to_html('data/execution/result.html')
df2.groupby(['testName', 'executionLogsId']).apply(summary).to_excel('data/execution/result_groupby.xlsx')
df2.groupby(['testName', 'executionLogsId']).agg({'executionLogsId': {'nunique'}, 'duration' : {'sum'}})
#df2.groupby(['testName']).apply(summary).to_html('data/execution/result.html') """

" def summary(x):\n    runs = x['executionLogsId'].nunique()\n    runs = float(runs)\n    duration = x['duration'].sum()/1000\n    mean_s = x['duration'].mean()/1000\n    count = duration/mean_s\n    result = {\n        'aggr_duration': duration,\n        'aggr_duration_minutes': duration/60,\n        'runs': runs,\n        'mean_sec': duration/runs,\n        'mean_minutes': duration/runs/60 ,\n        'steps': count\n    }\n    return pd.Series(result).round(1)\n\n#df2.groupby(['scriptName','sessionId']).apply(summary).to_html('data/execution/result.html')\ndf2.groupby(['testName', 'executionLogsId']).apply(summary).to_html('data/execution/result.html')\ndf2.groupby(['testName', 'executionLogsId']).apply(summary).to_excel('data/execution/result_groupby.xlsx')\ndf2.groupby(['testName', 'executionLogsId']).agg({'executionLogsId': {'nunique'}, 'duration' : {'sum'}})\n#df2.groupby(['testName']).apply(summary).to_html('data/execution/result.html') "

### Add a column to indicate Runs of each test script
As there is no other field available via the API, this field is generated each time the lineNum == 1.  This assumes that there will always be a step 1 in every test case.
Before this can be done, first sort the whole dataframe

In [ ]:
df2.sort_values(by=['executionLogsId','StepLogsId'], inplace=True)

In [ ]:
df2['run2'] = 0
run = 0
# Using DataFrame.index
for idx in df2.index:
    #print(df2['lineNum'][idx])
    if df2['lineNum'][idx] == 1:
        run = run + 1
    df2['run2'][idx] = run

/tmp/ipykernel_267/878714988.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['run2'][idx] = run


### Generate a field to indicate where test scripts have run to the last step of the script
This assumes that the last step is "End script"
First sort the dataframe by Runs
Then add a new text field that is a concatenation of all the step labels for each run
Finally, check which of those strings contain the text "End script"

In [ ]:
df2.sort_values(by=['executionLogsId','StepLogsId','run2'], inplace=True)

In [ ]:
df2['step_list'] = ''
df2['run_status'] = ''
df2['step_error'] = ''

In [ ]:
run = -1
for idx in df2.index:
    if run != df2['run2'][idx]:
        df2['step_list'][idx] = df2['StepLabel'][idx]
    else:
        df2['step_list'][idx] = df2['step_list'][idx] + df2['StepLabel'][idx]
    run == df2['run2'][idx]      
    
    if df2['step_status'][idx] == 'ERROR':
        df2['step_error'][idx] = 'ERROR'            
        

/tmp/ipykernel_267/3833809576.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['step_list'][idx] = df2['StepLabel'][idx]
/tmp/ipykernel_267/3833809576.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['step_error'][idx] = 'ERROR'


In [ ]:
df2

,topProjectName,testName,sessionId,executionLogsId,scriptId,scriptName,script_status,StepLabel,step_state,step_status,...,StepLogsId,takeScreenshots,screenshotsOnlyOnInterruption,duration_sec,duration_mins,date,run2,step_list,run_status,step_error
2273789,Integrated Performance Testing,User 8 - Send Load to Roadnet load for Planning,JSGa7oUWzM,2275,413,User 8 - Send Load to Roadnet load for Planning,ERROR,Start script,TERMINATED,SUCCESS,...,61738,False,False,17.308,0.288467,2023-01-26,1,Start script,Script run incomplete,
2273790,Integrated Performance Testing,User 8 - Send Load to Roadnet load for Planning,JSGa7oUWzM,2275,413,User 8 - Send Load to Roadnet load for Planning,ERROR,Go to Transportation management/Planning/Roadn...,TERMINATED,SUCCESS,...,61782,False,False,6.792,0.113200,2023-01-26,1,Go to Transportation management/Planning/Roadn...,Script run incomplete,
2273791,Integrated Performance Testing,User 8 - Send Load to Roadnet load for Planning,JSGa7oUWzM,2275,413,User 8 - Send Load to Roadnet load for Planning,ERROR,Click Routing lookup button,TERMINATED,SUCCESS,...,61801,False,False,0.640,0.010667,2023-01-26,1,Click Routing lookup button,Script run incomplete,
2273792,Integrated Performance Testing,User 8 - Send Load to Roadnet load for Planning,JSGa7oUWzM,2275,413,User 8 - Send Load to Roadnet load for Planning,ERROR,Select option DataSource / Sheet1 / Routing to...,TERMINATED,SUCCESS,...,61823,False,False,0.537,0.008950,2023-01-26,1,Select option DataSource / Sheet1 / Routing to...,Script run incomplete,
2273793,Integrated Performance Testing,User 8 - Send Load to Roadnet load for Planning,JSGa7oUWzM,2275,413,User 8 - Send Load to Roadnet load for Planning,ERROR,Click Warehouse,TERMINATED,SUCCESS,...,61828,False,False,0.550,0.009167,2023-01-26,1,Click Warehouse,Script run incomplete,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2274915,Integrated Performance Testing,Copy 1 - Receipted payment processing : SA Onl...,b3iiw3wwkr,2746,414,Copy 1 - Receipted payment processing : SA Onl...,NONE,Upload a file,RUNNING,SUCCESS,...,118638,False,False,4.256,0.070933,2023-01-26,609,Upload a file,Script run incomplete,
2274916,Integrated Performance Testing,Copy 1 - Receipted payment processing : SA Onl...,b3iiw3wwkr,2746,414,Copy 1 - Receipted payment processing : SA Onl...,NONE,Click Close,RUNNING,SUCCESS,...,118653,False,False,2.008,0.033467,2023-01-26,609,Click Close,Script run incomplete,
2274917,Integrated Performance Testing,Copy 1 - Receipted payment processing : SA Onl...,b3iiw3wwkr,2746,414,Copy 1 - Receipted payment processing : SA Onl...,NONE,Click Save,RUNNING,SUCCESS,...,118659,False,False,2.220,0.037000,2023-01-26,609,Click Save,Script run incomplete,
2274918,Integrated Performance Testing,Copy 1 - Receipted payment processing : SA Onl...,b3iiw3wwkr,2746,414,Copy 1 - Receipted payment processing : SA Onl...,NONE,Click Import,RUNNING,SUCCESS,...,118672,False,False,3.395,0.056583,2023-01-26,609,Click Import,Script run incomplete,


In [ ]:
df2['run_status'] = 'Script run incomplete'

In [ ]:
mask = (df2['step_list'].str.contains('End script'))
df2['run_status'].mask(mask,'Script completed',inplace=True)

### Add a column to indicate scripts where at least one step status is "Error".
This is necessary, as the script status is sometimes Success even where a step terminates in "Error".

### Generate an output file in CSV format

In [ ]:
df2.to_csv('data/execution/result.csv',index=False)

### From here on the script is disabled.

In [ ]:
# Insert this to prevent the cells below from trying to write to the sql table
raise SystemExit("Stop right here!")

SystemExit: Stop right here!

/home/wo/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


### Get ready to export to SQL

In [ ]:
mask = (df['date'] > '2023-01-10')
#df.loc[(df['date'] > '2023-01-10')]
df = df[mask]

In [ ]:
# https://learn.microsoft.com/en-us/sql/machine-learning/data-exploration/python-dataframe-pandas?view=sql-server-ver16
# Some other example server values are
# server = 'localhost\sqlexpress' # for a named instance
# server = 'myserver,port' # to specify an alternate port
server = 'tcp:csazniptlsql01.database.windows.net,1433' 
database = 'CSAZN-INT-IPT-L-DSQ-01' 
username = 'ccbaadmin' 
password = '@FireBase123' 
# ENCRYPT defaults to yes starting in ODBC Driver 18. It's good to always specify ENCRYPT=yes on the client side to avoid MITM attacks.
cnxn = pyodbc.connect('DRIVER={ODBC Driver 18 for SQL Server};SERVER='+server+';DATABASE='+database+';ENCRYPT=yes;UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

In [ ]:
# cursor = cnxn.cursor()

# cursor.execute('''
# 		CREATE TABLE ea_script_history (
# 			records_no int,
#             topProjectName nvarchar(255),
#             testName nvarchar(255),
#             sessionId nvarchar(255),
#             id int,
#             scriptId int,
#             scriptName nvarchar(255),
#             state nvarchar(255),
#             status nvarchar(255),
#             StepLabel nvarchar(255),
#             startTime datetime,
#             endTime datetime,
#             duration int,
#             lineNum int,
#             StepLogs_id int
# 			)
#                ''')

# cnxn.commit()

In [ ]:
cnxn = pyodbc.connect('DRIVER={ODBC Driver 18 for SQL Server};SERVER='+server+';DATABASE='+database+';ENCRYPT=yes;UID='+username+';PWD='+ password)
cursor = cnxn.cursor()
# Insert Dataframe into SQL Server:
i = 0
for index, row in df.iterrows():
     cursor.execute("INSERT INTO ea_script_history (records_no,topProjectName,testName,sessionId,id,scriptId,scriptName,state,status,StepLabel,startTime,endTime,duration,lineNum,StepLogs_id) values(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)", row.records_no, row.topProjectName, row.testName, row.sessionId, row.id, row.scriptId, row.scriptName, row.state, row.status, row.StepLabel, row.startTime, row.endTime, row.duration, row.lineNum, row.StepLogs_id)
     i = i + 1
     if i == 1000:
          i = 0
          cnxn.commit()          
cnxn.commit()
cursor.close()

In [ ]:
# cnxn = pyodbc.connect('DRIVER={ODBC Driver 18 for SQL Server};SERVER='+server+';DATABASE='+database+';ENCRYPT=yes;UID='+username+';PWD='+ password)
# cursor = cnxn.cursor()
# # Insert Dataframe into SQL Server:
# for index, row in df.iterrows():
#      cursor.execute("INSERT INTO ea_script_history (startTime) values(?)", row.startTime)
# cnxn.commit()
# cursor.close()

In [ ]:
# Delete all the rows in the SQl script history table
cursor = cnxn.cursor()
cursor.execute("DELETE FROM ea_script_history")
cnxn.commit()
cursor.close()

In [ ]:
# Drop the SQl script history table (it must be re-created)
cursor = cnxn.cursor()
cursor.execute("DROP Table ea_script_history")
#cursor.execute("DROP Table temp_table")
cnxn.commit()
cursor.close()

In [ ]:
line = '},        "tags": [],        "useDefaultEnvironment": true, }abc        "tags": []abc        "useDefaultEnvironment": true,'

line = re.sub(r"\},[^,]", "},\n", line) # OK
line = re.sub(r"\],[^,]", "],\n", line) # OK
line = re.sub(r"\}[^,]", "}\n", line) # OK
line = re.sub(r"\][^,]", "]\n", line) # OK


In [ ]:
print(line)